## <span style=color:blue>Illustrations of using python to load and manipulate Postgres data    </span>

<span style=color:blue>In particular, I downloaded the csv file "UNSD - Methodology.csv" from https://unstats.un.org/unsd/methodology/m49/overview/ into a local file, and in the code below, I create a table to hold it in my local postgres, upload the csv into the table, and then modify a few things</span>

In [1]:
# These are boiler plate imports that seem useful
# I should probably delete or comment out the ones that aren't used in this script...

import sys
import json
import csv
import yaml

import pandas as pd
import numpy as np

import matplotlib as mpl
from cycler import cycler

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid")

#sys.path.append('/Users/rick/HACKING/UTILITIES_IN_HACKING/')
# import utils_general

from owlready2 import *



## <span style=color:blue>Setting up Postgres connection.  Note database name is "foodsec_v02" </span>

In [12]:
# following https://www.geeksforgeeks.org/connecting-postgresql-with-sqlalchemy-in-python/

# for the sqlalchemy, added the "text" phrase following 
# https://stackoverflow.com/questions/75309237/read-sql-query-throws-optionengine-object-has-no-attribute-execute-with
# import psycopg2
from sqlalchemy import create_engine, text as sql_text

db_conn = create_engine('sqlite:///food.sqlite3',
                        isolation_level="AUTOCOMMIT")

# "isolation_level" was from second comment of
#    https://stackoverflow.com/questions/26717790/how-to-set-autocommit-1-in-a-sqlalchemy-engine-connection
#  I needed to include that, because for some SQL update commands the code
#    was executing against postgres, but then hanging, and I would have to kill the kernel
#    to free things up
#    (This was specifically for the ALTER TABLE command below switching from varchar to boolean data type)

### <span style=color:blue>Sanity check that I am accessing the database I want to </span>

In [13]:
q1 = '''
SELECT * 
FROM sqlite_schema
WHERE name = 'taxonomies'
'''

q2 = '''
select count(*)
from taxonomies
'''

# following pattern of
# https://stackoverflow.com/questions/75309237/read-sql-query-throws-optionengine-object-has-no-attribute-execute-with
df1 = pd.read_sql_query(con=db_conn.connect(),
                        sql=sql_text(q1))

df2 = pd.read_sql_query(con=db_conn.connect(),
                        sql=sql_text(q2))

print('Table listing')
print(df1.head())
print('\nCount of m49_geo_codes table')
print(df2.head())

Table listing
    type        name    tbl_name  rootpage   
0  table  taxonomies  taxonomies         3  \

                                                 sql  
0  CREATE TABLE "taxonomies" ("area_code" integer...  

Count of m49_geo_codes table
   count(*)
0        53


### <span style=color:blue>Creating table taxonomies.m49_geo_codes_test, and then loading from CSV file </span>

In [18]:
# when doing updates, remember to add "COMMIT;" command.  
# Otherwise your postgres will have a transaction that keeps running until you kill your kernel

q3 = '''
--DROP TABLE IF EXISTS taxonomies.m49_geo_codes_test;

CREATE TABLE IF NOT EXISTS m49_geo_codes_test (
  "Global Code" varchar(3),
  "Global Name" varchar,
  "Region Code" varchar(3),
  "Region Name" varchar,
  "Sub-region Code" varchar(3),
  "Sub-region Name" varchar,
  "Intermediate Region Code" varchar(3),
  "Intermediate Region Name" varchar,
  "Country or Area" varchar,
  "M49 Code" varchar(3),
  "ISO-alpha2 Code" varchar(2),
  "ISO-alpha3 Code" varchar(4),
  "Least Developed Countries (LDC)" varchar(1),
  "Land Locked Developing Countries (LLDC)" varchar(1),
  "Small Island Developing States (SIDS)" varchar(1)
  );

--COMMIT;
'''


# imitating
# https://stackoverflow.com/questions/75316741/attributeerror-engine-object-has-no-attribute-execute-when-trying-to-run-sq
# ALSO, need to wrap the 'q3' within 'sql_text(q3)'
with db_conn.connect() as conn:
    result = conn.execute(sql_text(q3))

print(result)

In [14]:
!pwd

/Users/claudio/projects/289l


In [19]:
# following https://www.postgresqltutorial.com/postgresql-tutorial/import-csv-file-into-posgresql-table/

csv_file = '/Users/claudio/projects/289l/UNSD_Methodology.csv'

q4 = '''
COPY taxonomies.m49_geo_codes_test (
  "Global Code",
  "Global Name",
  "Region Code",
  "Region Name",
  "Sub-region Code",
  "Sub-region Name",
  "Intermediate Region Code",
  "Intermediate Region Name",
  "Country or Area",
  "M49 Code",
  "ISO-alpha2 Code",
  "ISO-alpha3 Code",
  "Least Developed Countries (LDC)",
  "Land Locked Developing Countries (LLDC)",
  "Small Island Developing States (SIDS)"
)
FROM ''' + "'" + csv_file + "'" + '''
DELIMITER ';'
CSV HEADER;

--COMMIT;
'''

with db_conn.connect() as conn:
   result = conn.execute(sql_text(q4))

print(result)


OperationalError: (sqlite3.OperationalError) near "COPY": syntax error
[SQL: 
COPY taxonomies.m49_geo_codes_test (
  "Global Code",
  "Global Name",
  "Region Code",
  "Region Name",
  "Sub-region Code",
  "Sub-region Name",
  "Intermediate Region Code",
  "Intermediate Region Name",
  "Country or Area",
  "M49 Code",
  "ISO-alpha2 Code",
  "ISO-alpha3 Code",
  "Least Developed Countries (LDC)",
  "Land Locked Developing Countries (LLDC)",
  "Small Island Developing States (SIDS)"
)
FROM '/Users/claudio/projects/289l/UNSD_Methodology.csv'
DELIMITER ';'
CSV HEADER;

--COMMIT;
]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [20]:
# sanity check
q5 = '''
SELECT *
FROM m49_geo_codes_test'''

df5 = pd.read_sql_query(con=db_conn.connect(),
                        sql=sql_text(q5))

df5.head(10)

,Global Code,Global Name,Region Code,Region Name,Sub-region Code,Sub-region Name,Intermediate Region Code,Intermediate Region Name,Country or Area,M49 Code,ISO-alpha2 Code,ISO-alpha3 Code,Least Developed Countries (LDC),Land Locked Developing Countries (LLDC),Small Island Developing States (SIDS)
0,1,World,2.0,Africa,15.0,Northern Africa,,,Algeria,12,DZ,DZA,,,
1,1,World,2.0,Africa,15.0,Northern Africa,,,Egypt,818,EG,EGY,,,
2,1,World,2.0,Africa,15.0,Northern Africa,,,Libya,434,LY,LBY,,,
3,1,World,2.0,Africa,15.0,Northern Africa,,,Morocco,504,MA,MAR,,,
4,1,World,2.0,Africa,15.0,Northern Africa,,,Sudan,729,SD,SDN,x,,
5,1,World,2.0,Africa,15.0,Northern Africa,,,Tunisia,788,TN,TUN,,,
6,1,World,2.0,Africa,15.0,Northern Africa,,,Western Sahara,732,EH,ESH,,,
7,1,World,2.0,Africa,202.0,Sub-Saharan Africa,014,Eastern Africa,British Indian Ocean Territory,86,IO,IOT,,,
8,1,World,2.0,Africa,202.0,Sub-Saharan Africa,014,Eastern Africa,Burundi,108,BI,BDI,x,x,
9,1,World,2.0,Africa,202.0,Sub-Saharan Africa,014,Eastern Africa,Comoros,174,KM,COM,x,,x


### <span style=color:blue>Changing the boolean columns into actual Bool type    </span>

In [21]:
# following https://stackoverflow.com/questions/48877158/postgresql-9-4-alter-column-text-to-boolean-with-values

q6 = '''

ALTER TABLE m49_geo_codes_test
ALTER COLUMN "Least Developed Countries (LDC)" 
SET DATA TYPE boolean
using case
   when "Least Developed Countries (LDC)" = 'x' then true
   else false
end;

ALTER TABLE m49_geo_codes_test
ALTER COLUMN "Land Locked Developing Countries (LLDC)" 
SET DATA TYPE boolean
using case
   when "Land Locked Developing Countries (LLDC)" = 'x' then true
   else false
end;

ALTER TABLE m49_geo_codes_test
ALTER COLUMN "Small Island Developing States (SIDS)" 
SET DATA TYPE boolean
using case
   when "Small Island Developing States (SIDS)" = 'x' then true
   else false
end;

COMMIT;
'''

with db_conn.connect() as conn:
    result = conn.execute(sql_text(q6))

# sanity check
q9 = '''
SELECT *
FROM m49_geo_codes_test
'''

df9 = pd.read_sql_query(con=db_conn.connect(),
                        sql=sql_text(q9))

df9.head(10)


OperationalError: (sqlite3.OperationalError) near "ALTER": syntax error
[SQL: 

ALTER TABLE m49_geo_codes_test
ALTER COLUMN "Least Developed Countries (LDC)" 
SET DATA TYPE boolean
using case
   when "Least Developed Countries (LDC)" = 'x' then true
   else false
end;

ALTER TABLE m49_geo_codes_test
ALTER COLUMN "Land Locked Developing Countries (LLDC)" 
SET DATA TYPE boolean
using case
   when "Land Locked Developing Countries (LLDC)" = 'x' then true
   else false
end;

ALTER TABLE m49_geo_codes_test
ALTER COLUMN "Small Island Developing States (SIDS)" 
SET DATA TYPE boolean
using case
   when "Small Island Developing States (SIDS)" = 'x' then true
   else false
end;

COMMIT;
]
(Background on this error at: https://sqlalche.me/e/20/e3q8)